# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [0]:
# Importação dos pacotes para Analise
 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [0]:
#Carregar os dados para o google colab

uploaded = files.upload()

In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

### Análise dos dados

In [0]:
df['DESCRIÇÃO PARCEIRO'].str.slice(0,20)

0        PASTA INT VITAPOWER 
1        ESPONJA BETTANIN BRI
2        AGUA MIN SCHIN S/GAS
3        FITA DUPLA FACE C/SU
4        MASSA PIZZA ROMANHA 
                 ...         
22004    VINHO CHILENO ARESTI
22005    MOCHILA MASCULINA JU
22006    GEL FIX RADICAL PAST
22007    DOCE BANANA FALEX 16
22008    SHAMPOO TURMA XUXINH
Name: DESCRIÇÃO PARCEIRO, Length: 22009, dtype: object

### Etapa de Preparação do Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
df['DESC_AJUSTADA_SUB'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
# Dados de Teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
df2['DESC_AJUSTADA_SUB'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  df['DESC_AJUSTADA_SUB'] = df['DESC_AJUSTADA_SUB'].str.replace(x, ' ')
  # Dados de Teste
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')
  df2['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB'].str.replace(x, ' ')

In [0]:
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA,DESC_AJUSTADA_SUB
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C 1,ESPONJA BETTANIN BRILHUS C


In [0]:
# Variáveis importantes
# Palavras para retirar da análise
stop_words = ['de','do','da','kg','g','l','ml','e','gfa','un','lt','n','la','daiso',
              'acasa','massa','seara','friboi','big','granel','peca','montana',
              'temperado','chef','real','bananinha','bd','ceratti','sulita','macedo',
              'copacol','sadia','daucy','angus','pct','samurai','mezzani','fr','basilar',
              'colonial','fr','cesabela','nonna','rana','brasa','burguer','burguers',
              'perdigao','sodebo','atelie','mania','beef','germania','miguel','regency',
              'corttex','pompets','oceane','vermelho','azul','branco','verde','amarelo',
              'dona','clara','belem','sao','domingos','jorge','umbra','prisma','imp',
              'juta','lupo','urban','sport']
# Tipo acentuação
strip_accents = 'unicode'
# Tamanho da validação de teste
test_size = 0.1
# Interações no modelo
max_iter = 2000
# Tipo do vetor
tipo_vetor = TfidfVectorizer(stop_words=stop_words, strip_accents=strip_accents)

### Etapa 1 - Classificando DEPARTAMENTO

In [0]:
#Criando a nova para classificar DEPARTAMENTO
base = df
 
# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DESC_AJUSTADA']
base_target = base['DEPARTAMENTO']
 
# Transformando a descrição em vetor
vetor = tipo_vetor
vetor_x = vetor.fit_transform(base_data)
 
# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(vetor_x, base_target, test_size=test_size, random_state=27)
 
# Aqui nós configuramos o modelo para treinar o classificador
clf = PassiveAggressiveClassifier(max_iter=max_iter)

In [0]:
# Mostrar o tempo de processamento
%%time

# Executar o treinamento
clf.fit(x_train, y_train)

CPU times: user 1.07 s, sys: 742 ms, total: 1.81 s
Wall time: 939 ms


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=2000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [0]:
# Avaliando a performance com predição
predicted = clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                          precision    recall  f1-score   support

              AUTOMOTIVO       0.95      0.75      0.84        24
                 AÇOUGUE       0.94      0.96      0.95        53
      BEBIDAS ALCOÓLICAS       1.00      0.99      1.00       135
  BEBIDAS NÃO ALCOÓLICAS       0.95      0.91      0.93        68
               BRINQUEDO       1.00      0.81      0.90        16
                 CALÇADO       0.98      0.97      0.98        63
      CAMA, MESA E BANHO       0.98      1.00      0.99        44
        CONGELADOS DOCES       0.96      1.00      0.98        23
     CONGELADOS SALGADOS       0.80      0.67      0.73        12
    CUIDADOS COM A BARBA       1.00      1.00      1.00         3
     CUIDADOS COM A CASA       0.96      0.96      0.96        48
  CUIDADOS COM A COZINHA       1.00      0.91      0.95        22
    CUIDADOS COM A ROUPA       0.97      1.00      0.99        34
 CUIDADOS COM MÃOS E PÉS       0.95      

In [0]:
# Predição em novos dados com o DF  de teste
base_novos = df2['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n = vetor.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(vetor_n)

# Adicionando a coluna para o novo DF
df_dep = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_dep['DEPARTAMENTO'] = predicted_novos
df_dep['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_dep.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R


### Etapa 2 - Classificando CATEGORIA

In [0]:
# Criando a nova base para classificar CATEGORIA
base2 = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data2 = base2['DEPARTAMENTO']+' '+base2['DESC_AJUSTADA']
base_target2 = base2['CATEGORIA']

# Transformando a descrição em vetor
vetor2 = tipo_vetor
vetor_x2 = vetor2.fit_transform(base_data2)

# Dividindo dataset em treino e teste
x_train2, x_test2, y_train2, y_test2 = train_test_split(vetor_x2, base_target2, test_size=test_size, random_state=27)

In [0]:
# Mostrar o tempo de processamento
%%time

# Aqui nós configuramos o modelo para treinar o classificador
clf2 = PassiveAggressiveClassifier(max_iter=max_iter)

# Executar o treinamento
clf2.fit(x_train2, y_train2)

CPU times: user 6.26 s, sys: 4.3 s, total: 10.6 s
Wall time: 5.39 s


In [0]:
# Avaliando a performance com predição
predicted2 = clf2.predict(x_test2)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test2, predicted2))


###Indicadores Classificação Categoria###

                                           precision    recall  f1-score   support

                               ABSORVENTE       1.00      1.00      1.00         7
                      ACESSÓRIOS DE BANHO       1.00      1.00      1.00         4
                 ACESSÓRIOS DE LAVANDERIA       1.00      1.00      1.00         6
                 ACESSÓRIOS DE MANUTENÇÃO       0.50      0.50      0.50         2
                        ACESSÓRIOS ELETRO       1.00      1.00      1.00         3
                      ACESSÓRIOS MÃO E PÉ       0.91      0.77      0.83        13
                   ACESSÓRIOS PARA CABELO       0.86      1.00      0.92        12
                           ACESSÓRIOS PET       0.80      0.80      0.80         5
                                 ADOÇANTE       1.00      1.00      1.00         4
                       AGUARDENTE/CACHAÇA       1.00      1.00      1.00         3
                                  ALGODÃO 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Predição em novos dados utilizando o df_dep
base_novos2 = df_dep['DEPARTAMENTO']+' '+ df_dep['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n2 = vetor2.transform(base_novos2)

#Previsão dos novos dados
predicted_novos2 = clf2.predict(vetor_n2)

# Adicionando a coluna no novo DF
df_cat = df_dep
df_cat['CATEGORIA'] = predicted_novos2
df_cat['DESC_AJUSTADA_SUB'] = df_dep['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_cat.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL


### Etapa 3 - Classificando SUB-CATEGORIA

In [0]:
# Criando a nova para classificar CATEGORIA retirando apenas números
base3 = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data3 = base3['DEPARTAMENTO']+' '+base3['CATEGORIA']+' '+base3['DESC_AJUSTADA_SUB']
base_target3 = base3['SUB-CATEGORIA']

# Transformando a descrição em vetor
vetor3 = tipo_vetor
vetor_x3 = vetor3.fit_transform(base_data3)

# Dividindo dataset em treino e teste
x_train3, x_test3, y_train3, y_test3 = train_test_split(vetor_x3, base_target3, test_size=test_size, random_state=27)

In [0]:
# Mostrar o tempo de processamento
%%time

# Aqui nós treinamos o classificador
clf3 = PassiveAggressiveClassifier(max_iter=max_iter)

# Executar o treinamento
clf3.fit(x_train3, y_train3)

CPU times: user 38.1 s, sys: 26.2 s, total: 1min 4s
Wall time: 32.5 s


In [0]:
# Avaliando a performance com predição
predicted3 = clf3.predict(x_test3)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test3, predicted3))


###Indicadores Classificação Categoria###

                                          precision    recall  f1-score   support

                                  2 EM 1       0.00      0.00      0.00         0
                              40 VOLUMES       1.00      1.00      1.00         1
                                      4K       1.00      1.00      1.00         2
                                 ABACAXI       0.75      1.00      0.86         3
                               ABOBRINHA       0.00      0.00      0.00         2
                      ABOBRINHA ORGÂNICA       0.00      0.00      0.00         0
                                 ABÓBORA       1.00      1.00      1.00         2
                                  ACELGA       0.00      0.00      0.00         0
                         ACELGA ORGÂNICA       0.00      0.00      0.00         1
                               ACENDEDOR       1.00      1.00      1.00         1
                                 ACEROLA       0.00  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Predição em novos dados
base_novos3 = df_cat['DEPARTAMENTO']+' '+df_cat['CATEGORIA']+' '+df_cat['DESC_AJUSTADA_SUB']

# Tranformando descrição em vetor
vetor_n3 = vetor3.transform(base_novos3)

#Previsão dos novos dados
predicted_novos3 = clf3.predict(vetor_n3)

# Adicionando a coluna
df_sub = df_cat
df_sub['SUB-CATEGORIA'] = predicted_novos3

# Novos Dados de teste incluindo o departamento
df_sub.head(10)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA,SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL,SAL DO HIMALAIA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL,BOLA DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA,MERCEARIA DOCE,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,FARINÁCEO,MISTURAS
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,ACESSÓRIOS MÃO E PÉ,ALICATE
4,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,MEIA PIZZA PEPPERONI UN,PIZZA,PEPPERONI
5,JARRA ACASA VIDRO CURVES OJG378,JARRA ACASA VIDRO CURVES OJG,UTILIDADES DE COZINHA,JARRA ACASA VIDRO CURVES OJG,UTENSÍLIOS DE VIDRO,JARRA/GARRAFA
6,FILME DE PVC CONSERV PACK 30M,FILME DE PVC CONSERV PACK,DESCARTÁVEIS,FILME DE PVC CONSERV PACK M,FILME PVC,FILME PVC
7,CESTO VIME CRAFTWORK S-25X15XH13,CESTO VIME CRAFTWORK S,LAVANDERIA,CESTO VIME CRAFTWORK S XXH,ORGANIZADOR,CESTO DE ROUPA
8,MAC BORTOLINI 500GR OVOS ESPAGUETE .,MAC BORTOLINI,MERCEARIA SALGADA,MAC BORTOLINI GR OVOS ESPAGUETE,MASSA SECA,ESPAGUETE
9,COND NOVEX OLEO OJON 300ML,COND NOVEX OLEO OJON,CUIDADOS COM O CABELO,COND NOVEX OLEO OJON ML,CONDICIONADOR,TRADICIONAL


In [0]:
dotz = df_sub[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')

In [0]:
df.query('DEPARTAMENTO == "AÇOUGUE"')

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA,DESC_AJUSTADA_SUB
21,BISTECA BOV HEREFORD EG KG,BISTECA,BOVINO RESFRIADO,AÇOUGUE,BISTECA BOV HEREFORD EG KG,BISTECA BOV HEREFORD EG KG
90,ALCATRA SUIN DATERRA GRILL RESF KG,ALCATRA,BOVINO RESFRIADO,AÇOUGUE,ALCATRA SUIN DATERRA GRILL RESF KG,ALCATRA SUIN DATERRA GRILL RESF KG
92,COXA ASA APIM B FRANGO 1kg,COXA,AVE RESFRIADA,AÇOUGUE,COXA ASA APIM B FRANGO,COXA ASA APIM B FRANGO kg
94,CUPIM PC.INT.VACUO KG,CUPIM,BOVINO RESFRIADO,AÇOUGUE,CUPIM PC INT VACUO KG,CUPIM PC INT VACUO KG
108,MUSCULO TRAS BOV DISTRIBOI (PECA) KG,MUSCULO,BOVINO RESFRIADO,AÇOUGUE,MUSCULO TRAS BOV DISTRIBOI PECA KG,MUSCULO TRAS BOV DISTRIBOI PECA KG
...,...,...,...,...,...,...
21908,LAGARTO (TATU) MOIDO BOV ARGUS (P) KG,LAGARTO,BOVINO RESFRIADO,AÇOUGUE,LAGARTO TATU MOIDO BOV ARGUS P KG,LAGARTO TATU MOIDO BOV ARGUS P KG
21914,PEITO FGO C/OSSO kg CONG,PEITO DE FRANGO,AVE CONGELADA,AÇOUGUE,PEITO FGO C OSSO kg CONG,PEITO FGO C OSSO kg CONG
21921,PEITO BOVINO C/OSSO kg,PEITO BOVINO,BOVINO CONGELADO,AÇOUGUE,PEITO BOVINO C OSSO kg,PEITO BOVINO C OSSO kg
21928,PALETA BOV MONTANA (PEDACO) (P) KG,PALETA,BOVINO RESFRIADO,AÇOUGUE,PALETA BOV MONTANA PEDACO P KG,PALETA BOV MONTANA PEDACO P KG


In [0]:
df_sub.query('DEPARTAMENTO == "AÇOUGUE"')

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA,SUB-CATEGORIA
98,COXAO MOLE BROCHETE KG,COXAO MOLE BROCHETE KG,AÇOUGUE,COXAO MOLE BROCHETE KG,BOVINO RESFRIADO,COXÃO MOLE
113,FRANGO RESFRIADO kg SEARA,FRANGO RESFRIADO kg SEARA,AÇOUGUE,FRANGO RESFRIADO kg SEARA,AVE RESFRIADA,AVE INTEIRA
148,LINGUA BOV ELGOLLI (PECA) KG,LINGUA BOV ELGOLLI PECA KG,AÇOUGUE,LINGUA BOV ELGOLLI PECA KG,BOVINO RESFRIADO,LÍNGUA BOVINA
173,PERNIL SUINO kg CONGELADO,PERNIL SUINO kg CONGELADO,AÇOUGUE,PERNIL SUINO kg CONGELADO,SUÍNO RESFRIADO,PERNIL
414,AVE FIESTA TEMP.SEARA ASSA FACIL KG,AVE FIESTA TEMP SEARA ASSA FACIL KG,AÇOUGUE,AVE FIESTA TEMP SEARA ASSA FACIL KG,AVE RESFRIADA,AVE INTEIRA
...,...,...,...,...,...,...
9151,COXAO MOLE COZIDO ORIENTAL KG,COXAO MOLE COZIDO ORIENTAL KG,AÇOUGUE,COXAO MOLE COZIDO ORIENTAL KG,BOVINO RESFRIADO,COXÃO MOLE
9160,PALETA SUINA RAINHA DA PAZ RESF (P) KG,PALETA SUINA RAINHA DA PAZ RESF P KG,AÇOUGUE,PALETA SUINA RAINHA DA PAZ RESF P KG,SUÍNO RESFRIADO,PALETA
9311,PALETA BOV FRIBOI (PEDACO) KG,PALETA BOV FRIBOI PEDACO KG,AÇOUGUE,PALETA BOV FRIBOI PEDACO KG,BOVINO RESFRIADO,PALETA
9325,FILE PEITO FRANGO NOBRE RESF BJ 700G,FILE PEITO FRANGO NOBRE RESF BJ,AÇOUGUE,FILE PEITO FRANGO NOBRE RESF BJ G,AVE RESFRIADA,FILÉ
